## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [13]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Gravdal,NO,68.1214,13.5031,33.80,80,75,19.57,sleet
1,1,Bethel,US,41.3712,-73.4140,33.80,34,90,10.36,overcast clouds
2,2,Tuktoyaktuk,CA,69.4541,-133.0374,-20.20,76,75,4.61,broken clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,51.80,62,40,18.41,scattered clouds
4,4,Chokurdakh,RU,70.6333,147.9167,-21.15,100,93,3.53,overcast clouds


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 105


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Ushuaia,AR,-54.8000,-68.3000,51.80,62,40,18.41,scattered clouds
5,5,Hobart,AU,-42.8794,147.3294,62.60,63,0,13.80,clear sky
6,6,Bowen,AU,-20.0167,148.2333,68.00,88,40,8.05,light rain
7,7,Hilo,US,19.7297,-155.0900,73.99,68,1,4.61,clear sky
8,8,Butaritari,KI,3.0707,172.7902,81.19,81,44,16.51,light rain
13,13,Mitsamiouli,KM,-11.3847,43.2844,82.42,77,27,10.47,light rain
14,14,Chabahar,IR,25.2919,60.6430,80.46,60,0,9.35,clear sky
15,15,Castro,BR,-24.7911,-50.0119,72.03,47,1,2.37,clear sky
16,16,New Norfolk,AU,-42.7826,147.0587,62.60,59,0,13.80,clear sky
17,17,Avera,US,33.1940,-82.5271,53.60,28,1,10.36,clear sky


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()
#preferred_cities_df.count()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Ushuaia,AR,51.80,scattered clouds,-54.8000,-68.3000,
5,Hobart,AU,62.60,clear sky,-42.8794,147.3294,
6,Bowen,AU,68.00,light rain,-20.0167,148.2333,
7,Hilo,US,73.99,clear sky,19.7297,-155.0900,
8,Butaritari,KI,81.19,light rain,3.0707,172.7902,
13,Mitsamiouli,KM,82.42,light rain,-11.3847,43.2844,
14,Chabahar,IR,80.46,clear sky,25.2919,60.6430,
15,Castro,BR,72.03,clear sky,-24.7911,-50.0119,
16,New Norfolk,AU,62.60,clear sky,-42.7826,147.0587,
17,Avera,US,53.60,clear sky,33.1940,-82.5271,


In [20]:
# 6a. Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df[(hotel_df["Hotel Name"] == "")].index)
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Ushuaia,AR,51.80,scattered clouds,-54.8000,-68.3000,Albatros Hotel
5,Hobart,AU,62.60,clear sky,-42.8794,147.3294,Mantra on Collins Hobart
6,Bowen,AU,68.00,light rain,-20.0167,148.2333,Castle Motor Lodge Motel
7,Hilo,US,73.99,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
8,Butaritari,KI,81.19,light rain,3.0707,172.7902,Isles Sunset Lodge
13,Mitsamiouli,KM,82.42,light rain,-11.3847,43.2844,Foyer ADM
14,Chabahar,IR,80.46,clear sky,25.2919,60.6430,هتل گدروشیا
15,Castro,BR,72.03,clear sky,-24.7911,-50.0119,CHACARA BAILLY
16,New Norfolk,AU,62.60,clear sky,-42.7826,147.0587,The Shingles Riverside Cottages
19,Kaoma,ZM,70.63,overcast clouds,-14.7833,24.8000,KAOMA AIRPORT


In [25]:
# 8a. Create the output File (CSV)
output_data_file = "../Weather_Database/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '2px solid black',
    'padding': '.5px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(border='2px solid black', height='600px', margin='0 auto 0 auto', padding='.5px', w…